<a href="https://colab.research.google.com/github/sp2743/datamining-samplequestion/blob/main/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Task 1: Generate 4 random transaction datasets for experiment          [10]

---


    # transactions in each dataset: 20
    # items per transaction: range 1-7  [Repreation not allowed]
    # items set:- {i1, i2, ------------------- i40}
    
    Example:-
    T1: {i3, i1, i5}
    T2: {i6, i12, i3, i40, i11, i9, i33}

In [7]:
import random
import pandas as pd

In [3]:
import random

def generate_transactions():
    """Generates random transactions for a dataset."""
    items = [f"i{x}" for x in range(1, 41)]  # Items {i1, i2, ..., i40}
    transactions = []
    for i in range(1, 21):  # 20 transactions per dataset
        transaction = random.sample(items, random.randint(1, 7))
        transactions.append(f"T{i}: {{{', '.join(transaction)}}}")
    return transactions

# Generate and save four datasets
for dataset_num in range(1, 5):
    transactions = generate_transactions()
    with open(f"dataset_{dataset_num}.txt", "w") as file:
        file.write("\n".join(transactions))

def read_and_print_datasets(num_datasets=4):
    """Reads datasets from files and prints them in the specified format."""
    for dataset_num in range(1, num_datasets + 1):
        print(f"dataset{dataset_num}:")

        file_path = f"dataset_{dataset_num}.txt"
        with open(file_path, "r") as file:
            for line in file:
                print(line.strip().replace(": ", "-"))

        print("-" * 40)

# Run the function
read_and_print_datasets()


dataset1:
T1-{i27, i6}
T2-{i7, i26, i27, i11, i25}
T3-{i18, i6, i12}
T4-{i31, i11, i27}
T5-{i39}
T6-{i18, i12, i6, i38}
T7-{i3, i18, i37, i9, i24}
T8-{i17, i19, i14, i27, i18, i13}
T9-{i12}
T10-{i5}
T11-{i27, i3, i29}
T12-{i3, i12, i33, i11, i18}
T13-{i5, i18, i16, i17, i36}
T14-{i28, i17, i39, i32, i12}
T15-{i32, i23, i33, i12, i11, i22}
T16-{i20}
T17-{i37, i24, i21, i32, i26, i18}
T18-{i39}
T19-{i12, i7, i8, i17, i20}
T20-{i12, i17, i31, i33}
----------------------------------------
dataset2:
T1-{i24, i7, i26, i23, i25}
T2-{i16, i37, i28, i31, i6, i32}
T3-{i3, i21, i13, i6, i33, i35}
T4-{i19, i28}
T5-{i24, i33}
T6-{i34}
T7-{i2, i11, i30, i21, i35}
T8-{i5, i17, i27, i38, i33, i10, i3}
T9-{i26}
T10-{i17, i8, i7, i32, i28, i20}
T11-{i6, i2, i3, i36, i34}
T12-{i10, i5, i33, i24, i9, i27, i11}
T13-{i25, i36}
T14-{i35, i36, i12, i4, i39, i30, i10}
T15-{i37, i40, i28, i22, i3}
T16-{i17, i16, i36, i34, i3, i11}
T17-{i28, i13, i21}
T18-{i21, i25, i30, i12, i37, i9}
T19-{i20, i25, i21, i10, i4

### Task 2: Implementing Apriori Algorithm from Scratch         [20]

Write a Python program to implement the Apriori algorithm to find frequent itemsets and generate association rules from a given transaction dataset.


    Accept transactions as input (manually or from a file).
    Set a minimum support threshold and confidence threshold.
    Implement candidate generation, pruning, and rule generation.
    Display frequent itemsets and association rules.

In [7]:
def load_transactions(file_path):
    """Loads transactions from a file and returns a list of sets."""
    transactions = []
    with open(file_path, "r") as file:
        for line in file:
            items = line.strip().split(": ")[1]  # Extract items after "T1: "
            items = items.strip("{}").split(", ")  # Remove {} and split items
            transactions.append(set(items))
    return transactions

In [8]:
import itertools

def get_frequent_itemsets(transactions, min_support):
    """Finds frequent itemsets with given min_support."""
    item_count = {}
    total_transactions = len(transactions)

    for transaction in transactions:
        for item in transaction:
            item_count[item] = item_count.get(item, 0) + 1

    return {frozenset([item]): count for item, count in item_count.items() if count / total_transactions >= min_support}

def new_candidates(frequent_itemsets):
    """Generates new candidate itemsets from frequent itemsets."""
    candidates = {}
    items = list({item for itemset in frequent_itemsets for item in itemset})

    for i in range(len(items)):
        for j in range(i+1, len(items)):
            new_item = frozenset([items[i], items[j]])
            if new_item not in candidates:
                candidates[new_item] = 0
    return candidates


def prune_candidates(candidates, frequent_itemsets):
    """Prunes candidates that have infrequent subsets."""
    pruned_candidates = {}
    for candidate in candidates:
        if all(frozenset([item]) in frequent_itemsets for item in candidate):
            pruned_candidates[candidate] = 0
    return pruned_candidates

def generate_association_rules(frequent_itemsets, min_confidence):
    """Generates association rules from frequent itemsets."""
    rules = []
    for itemset, support in frequent_itemsets.items():
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                for antecedent in itertools.combinations(itemset, i):
                    antecedent = frozenset(antecedent)
                    consequent = itemset - antecedent
                    if consequent and antecedent in frequent_itemsets:
                        confidence = support / frequent_itemsets[antecedent]
                        if confidence >= min_confidence:
                            rules.append((antecedent, consequent, confidence))
    return rules

In [9]:
file_path = "dataset_1.txt"
transactions = load_transactions(file_path)

min_support = 0.2
min_confidence = 0.5

# Find frequent itemsets
frequent_itemsets = get_frequent_itemsets(transactions, min_support)

print("\nFrequent Itemsets:")
for itemset, support in frequent_itemsets.items():
    print(f"{set(itemset)}: Support = {support}")


candidates = new_candidates(frequent_itemsets)
pruned_candidates = prune_candidates(candidates, frequent_itemsets)


rules = generate_association_rules(pruned_candidates, min_confidence)

print("\nAssociation Rules:")
for antecedent, consequent, confidence in rules:
    print(f"Rule: {set(antecedent)} -> {set(consequent)}, Confidence = {confidence:.2f}")



Frequent Itemsets:
{'i27'}: Support = 5
{'i11'}: Support = 4
{'i12'}: Support = 8
{'i18'}: Support = 7
{'i17'}: Support = 5

Association Rules:


### Task 3: Applying Apriori using Python Libraries (MLxtend)   [20]
Use the MLxtend library to find frequent itemsets and generate association rules.

    Use mlxtend.frequent_patterns.apriori to extract frequent itemsets.
    Use mlxtend.frequent_patterns.association_rules to generate association rules.
    
    For more details:- https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

In [10]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

def encode_transactions(transactions):
    """Encodes transactions into a DataFrame for MLxtend's Apriori."""
    unique_items = sorted({item for transaction in transactions for item in transaction})
    encoded_data = [{item: (item in transaction) for item in unique_items} for transaction in transactions]
    return pd.DataFrame(encoded_data)

def run_apriori_mlxtend(df, min_support, min_confidence):
    """Runs MLxtend's Apriori algorithm."""
    frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)
    if frequent_itemsets.empty:
        return 0, 0
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    return len(frequent_itemsets), len(rules)

In [13]:
file_path = "dataset_1.txt"
transactions = load_transactions(file_path)


df = encode_transactions(transactions)


min_support = 0.2
min_confidence = 0.5

num_frequent_itemsets, num_rules = run_apriori_mlxtend(df, min_support, min_confidence)


print(f"\nTotal Frequent Itemsets: {num_frequent_itemsets}")
print(f"Total Association Rules: {num_rules}")

# Obtain frequent itemsets and rules using apriori and association_rules
frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)
if not frequent_itemsets.empty:
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

print("\nFrequent Itemsets:")
print(frequent_itemsets)

if rules is not None and not rules.empty:
    print("\nAssociation Rules:")
    print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


Total Frequent Itemsets: 5
Total Association Rules: 0

Frequent Itemsets:
   support itemsets
0     0.20    (i11)
1     0.40    (i12)
2     0.25    (i17)
3     0.35    (i18)
4     0.25    (i27)


### Task 4: Experimenting with Different Support & Confidence Levels [Self-Implemented & In-Built]

Analyze how changing minimum support and confidence thresholds affects the number of frequent itemsets and association rules.

For each dataset:-

    Run the Apriori algorithm with different min_support and min_confidence values:-          [20]
    (0.2, 0.4),  (0.5, 0.7), (0.8, 0.9)
    
    Compare the number of frequent itemsets and association rules generated (tabular format). [10]
    
    
#### Write final observations on how the results change. [20]

In [12]:
support_confidence_pairs = [(0.2, 0.4), (0.5, 0.7), (0.8, 0.9)]
results = []

for dataset_num in range(1, 5):
    file_path = f"dataset_{dataset_num}.txt"
    transactions = load_transactions(file_path)
    df = encode_transactions(transactions)

    for min_support, min_confidence in support_confidence_pairs:
        # Run Manual Apriori
        frequent_itemsets = get_frequent_itemsets(transactions, min_support)
        candidates = new_candidates(frequent_itemsets)
        pruned_candidates = prune_candidates(candidates, frequent_itemsets)
        rules = generate_association_rules(pruned_candidates, min_confidence)

        # Run MLxtend Apriori
        mlxtend_frequent, mlxtend_rules = run_apriori_mlxtend(df, min_support, min_confidence)

        # Store results
        results.append([dataset_num, min_support, min_confidence, len(pruned_candidates), len(rules), mlxtend_frequent, mlxtend_rules])

# Convert results to DataFrame and save
results_df = pd.DataFrame(results, columns=["Dataset", "Min Support", "Min Confidence",
                                            "Manual Frequent Itemsets", "Manual Rules",
                                            "MLxtend Frequent Itemsets", "MLxtend Rules"])
results_df.to_csv("apriori_results.csv", index=False)

# Print Summary
print("\nComparison of Manual vs MLxtend Apriori Results:")
print(results_df.to_markdown())


Comparison of Manual vs MLxtend Apriori Results:
|    |   Dataset |   Min Support |   Min Confidence |   Manual Frequent Itemsets |   Manual Rules |   MLxtend Frequent Itemsets |   MLxtend Rules |
|---:|----------:|--------------:|-----------------:|---------------------------:|---------------:|----------------------------:|----------------:|
|  0 |         1 |           0.2 |              0.4 |                         10 |              0 |                           5 |               0 |
|  1 |         1 |           0.5 |              0.7 |                          0 |              0 |                           0 |               0 |
|  2 |         1 |           0.8 |              0.9 |                          0 |              0 |                           0 |               0 |
|  3 |         2 |           0.2 |              0.4 |                         45 |              0 |                          10 |               0 |
|  4 |         2 |           0.5 |              0.7 |         

## Analysis
### Higher Support Thresholds Reduce Frequent Itemsets
### Higher Confidence Thresholds Reduce
### Low confidence (0.4) allows more association rules, while high confidence (0.7 or 0.9) often prevents any rule generation